# Setting up

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'Lab/Grain-U-Net'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))
sys.path.append('/content/drive/MyDrive/{}/src'.format(FOLDERNAME))
%cd /content/drive/My\ Drive/$FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive/Lab/Grain-U-Net


In [2]:
%load_ext autoreload
%autoreload 2
%aimport -tensorflow
!pip install bayesian-optimization  
!pip install -q plantcv
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=1912253ff5b3104140a2a1987f38ecf199d58b5e231e7c4a3becb5ca679f3efd
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization
     |████████████████████████████████| 285 kB 5.0 MB/s 
     |████████████████████████████████| 40 kB 5.3 MB/s 
     |████████████████████████████████| 837 kB 43.0 MB/s 
     |████████████████████████████████| 1.1 MB 58.0 MB/s 
     |████████████████████████████████| 140 kB 63.3 MB/s 
     |████████████████████████████████| 596 kB 61.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but yo

# Impoting Data

In [3]:
from pathlib import Path
import numpy as np

train_dir = Path('data/train_atarashii_256')

input_names = list(train_dir.glob('**/image/*.png'))
label_names = list(train_dir.glob('**/label/*.png'))

print(f"Found {len(input_names)} samples and {len(label_names)} tracings")

Found 675 samples and 675 tracings


In [4]:
from src import GrainSequence
import random

def createSequences(batch_size): 
    validation_samples = len(input_names) // 10

    random.Random(1337).shuffle(input_names)
    random.Random(1337).shuffle(label_names)

    train_input = input_names[:-validation_samples]
    train_label = label_names[:-validation_samples]
    train_gen = GrainSequence(batch_size, (256,256), train_input, train_label)

    valid_input = input_names[-validation_samples:]
    valid_label = label_names[-validation_samples:]
    valid_gen = GrainSequence(batch_size, (256,256), valid_input, valid_label)

    print(f"Training set size: {len(train_input)}, {len(train_gen)} batches")
    print(f"Validation set size: {len(valid_input)}, {len(valid_gen)} batches")

    return [train_gen, valid_gen]

# Creating the model

In [5]:
import numpy as np 
from keras.optimizers import *
from src import get_unet_hyper

def fit_with(dropout_rate, lr, epochs, batch_size, verbose=1):
    epochs = int(epochs)
    batch_size = int(batch_size)
    model = get_unet_hyper(dropout_rate=dropout_rate)

    l = createSequences(batch_size)
    train_gen = l[0]
    valid_gen = l[1]

    opt = keras.optimizers.Adam(learning_rate = lr)
    model.compile(optimizer = opt, loss='binary_crossentropy', metrics = ['accuracy'])

    model.fit(x=train_gen, steps_per_epoch=len(train_gen), epochs=epochs, batch_size=batch_size, verbose=verbose)

    #add in test set
    score=model.evaluate(valid_gen , steps=len(valid_gen), verbose=0)

    print('Test loss: ', score[0])
    print('Test accuracy: ', score[1])

    return score[1]

In [6]:
from tensorflow import keras
from bayes_opt import BayesianOptimization
import functools

#dropout_rate, lr, epochs, batch_size
pbounds = {'dropout_rate': (0.3, 0.8), 
           'lr': (1e-4, 1e-3), 
           'epochs': (25, 40), 
           'batch_size': (1,16)
           }


In [7]:
optimizer = BayesianOptimization(
    f=fit_with, 
    pbounds=pbounds, 
    verbose=2, 
    random_state=1
)

optimizer.maximize(init_points=4, n_iter=10)

for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print(optimizer.max)

|   iter    |  target   | batch_... | dropou... |  epochs   |    lr     |
-------------------------------------------------------------------------
Training set size: 608, 86 batches
Validation set size: 67, 9 batches
Epoch 1/25
86/86 [==============================] - 215s 2s/step - loss: 0.3825 - accuracy: 0.9512
Epoch 2/25
86/86 [==============================] - 44s 511ms/step - loss: 0.1977 - accuracy: 0.9516
Epoch 3/25
86/86 [==============================] - 44s 505ms/step - loss: 0.1944 - accuracy: 0.9516
Epoch 4/25
86/86 [==============================] - 44s 508ms/step - loss: 0.1944 - accuracy: 0.9516
Epoch 5/25
86/86 [==============================] - 44s 507ms/step - loss: 0.1940 - accuracy: 0.9516
Epoch 6/25
86/86 [==============================] - 44s 507ms/step - loss: 0.1940 - accuracy: 0.9516
Epoch 7/25
86/86 [==============================] - 44s 507ms/step - loss: 0.1942 - accuracy: 0.9516
Epoch 8/25
86/86 [==============================] - 44s 507ms/step - loss: 0.